<a href="https://colab.research.google.com/github/SegunDave/BybitCCxt/blob/main/ByBitData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip -q install ccxt pandas_ta schedule plotly==5.4.0

In [2]:
import ccxt,pytz,time,schedule, requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import pandas_ta as ta
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import clear_output
import matplotlib.pyplot as plt
#from binance.client import Client
from pprint import pprint
import statsmodels.regression.linear_model as rg



In [ ]:
bybit = ccxt.bybit()

instrument = 'MATIC/USDT'
Tframe ='15m'
lim = 1000

# get BTCUSDT 1 hr kline data
klines = bybit.fetch_ohlcv(instrument, timeframe=Tframe, limit= lim, since=None)

# filter klines to only include data from the past month
from datetime import datetime, timedelta
one_month_ago = datetime.now() - timedelta(days=30)
filtered_klines = [kline for kline in klines if datetime.fromtimestamp(kline[0]/1000) >= one_month_ago]

print(filtered_klines)


In [ ]:

# convert the klines list to a DataFrame
df = pd.DataFrame(filtered_klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

# convert the timestamp column to a datetime type
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# set the timestamp column as the index
df.set_index('timestamp', inplace=True)

print(df)


In [ ]:
import plotly.graph_objs as go

# create a line chart using the close data
line = go.Scatter(
    x=df.index,
    y=df['close'],
    name='Close',
    line=dict(color='#17BECF')
)

# create the layout for the chart
layout = go.Layout(
    title= f'{instrument} in {Tframe} Kline Data',
    xaxis=dict(
        title='Time',
        rangeslider=dict(visible=False)
    ),
    yaxis=dict(
        title='Price (USDT)'
    )
)

# create the figure and plot the chart
fig = go.Figure(data=[line], layout=layout)
fig.show()

# calculate RSI
df['rsi'] = ta.rsi(df['close'])

# calculate Stochastics
df['stoch_k'], df['stoch_d'] = ta.stoch(df['high'], df['low'], df['close'])


In [15]:
import plotly.express as px

# create the line chart
fig = px.line(df, x=df.index, y=df['rsi'], title=f'{instrument} in {Tframe} Kline Data')

# add the red and green lines
fig.add_shape(
    type='line',
    x0=df.index[0],
    x1=df.index[-1],
    y0=75,
    y1=75,
    yref='y',
    line=dict(color='red', dash='dot')
)
fig.add_shape(
    type='line',
    x0=df.index[0],
    x1=df.index[-1],
    y0=25,
    y1=25,
    yref='y',
    line=dict(color='green', dash='dot')
)

# set the y-axis range to include 0 and 100
fig.update_layout(yaxis=dict(range=[0, 100]))

fig.show()
